# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd 
import requests 
import xlsxwriter 
import math
from scipy import stats 
import configparser

In [2]:
# Loading keys from config parser
config = configparser.ConfigParser()
config.read_file(open('/home/venom/GitHub/cheatsheets/courses/freecodecamp/algorithmic_trading_with_python/secret.cfg'))

In [3]:
IEX_SANDBOX_PRIVATE = config.get('IEX', 'IEX_SANDBOX_PRIVATE')
IEX_SANDBOX_PUBLIC = config.get('IEX', 'IEX_SANDBOX_PUBLIC')

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [4]:
stocks = pd.read_csv('/home/venom/GitHub/cheatsheets/courses/freecodecamp/algorithmic_trading_with_python/data/sp_500_stocks.csv')

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [6]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_SANDBOX_PUBLIC}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'TACAEEEAGQLMOSA SNTNLG)S/ KBR(DL ',
 'calculationPrice': 'tops',
 'open': None,
 'openTime': None,
 'openSource': 'califfio',
 'close': None,
 'closeTime': None,
 'closeSource': 'flfaioci',
 'high': None,
 'highTime': None,
 'highSource': None,
 'low': None,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 141.545,
 'latestSource': 'IEX real time price',
 'latestTime': '1:06:08 PM',
 'latestUpdate': 1622250458309,
 'latestVolume': None,
 'iexRealtimePrice': 139.594,
 'iexRealtimeSize': 103,
 'iexLastUpdated': 1617254403706,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 135.2,
 'previousVolume': 107022446,
 'change': 4.255,
 'changePercent': 0.032,
 'volume': None,
 'iexMarketPercent': 0.018913532070779917,
 'iexVolume': 140

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [7]:
pe_ratio = data['peRatio']
pe_ratio

41.83

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [8]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [9]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_SANDBOX_PUBLIC}'
    
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['peRatio'],
                    'N/A'
                ],
                index=my_columns
            ),
            ignore_index=True
        )

In [10]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,127.110,57.39,N/A
1,AAL,16.363,-1.17,N/A
2,AAP,170.640,25,N/A
3,AAPL,142.626,42.4,N/A
4,ABBV,115.050,25.12,N/A
...,...,...,...,...
500,YUM,110.760,32.44,N/A
501,ZBH,168.380,-229.4,N/A
502,ZBRA,414.110,46.85,N/A
503,ZION,49.160,20.96,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [12]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
340,NRG,41.390,2.5,N/A
335,NLOK,21.312,4,N/A
19,AIV,4.850,4.63,N/A
63,BIO,593.490,5.19,N/A
459,UNM,26.810,5.73,N/A
...,...,...,...,...
202,GLW,39.746,174.23,N/A
101,CMG,1508.070,181.77,N/A
247,IQV,195.179,212.33,N/A
296,MAR,128.000,242.9,N/A


In [13]:
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe.drop('index', axis=1, inplace=True)

/home/venom/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [14]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,41.390,2.5,N/A
1,NLOK,21.312,4,N/A
2,AIV,4.850,4.63,N/A
3,BIO,593.490,5.19,N/A
4,UNM,26.810,5.73,N/A
5,AFL,46.896,7.62,N/A
6,EBAY,57.250,7.87,N/A
7,ALL,113.442,8,N/A
8,CPB,48.467,8.03,N/A
9,KIM,17.149,8.29,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [15]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [16]:
portfolio_input()

Enter the value of your portfolio: 10000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [17]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

/home/venom/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,41.390,2.5,4832
1,NLOK,21.312,4,9384
2,AIV,4.850,4.63,41237
3,BIO,593.490,5.19,336
4,UNM,26.810,5.73,7459
5,AFL,46.896,7.62,4264
6,EBAY,57.250,7.87,3493
7,ALL,113.442,8,1763
8,CPB,48.467,8.03,4126
9,KIM,17.149,8.29,11662


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [19]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_SANDBOX_PUBLIC}'

data = requests.get(batch_api_call_url).json()

# P/E ratio 
pe_ratio = data[symbol]['quote']['peRatio']
# P/B ratio 
pb_ratio = data[symbol]['advanced-stats']['priceToBook']
# P/S ratio 
ps_ratio = data[symbol]['advanced-stats']['priceToSales']
# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda
# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [20]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns=rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_SANDBOX_PUBLIC}'
    
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

In [21]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,128.912,N/A,57.81,N/A,8.1,N/A,7.67,N/A,33.690997,N/A,13.806188,N/A,N/A
1,AAL,15.884,N/A,-1.16,N/A,-1.77,N/A,0.4111,N/A,-5.919055,N/A,1.379237,N/A,N/A
2,AAP,169.186,N/A,25.23,N/A,3.04,N/A,1.17,N/A,11.500474,N/A,2.485614,N/A,N/A
3,AAPL,139.550,N/A,43.5,N/A,34.81,N/A,8.18,N/A,29.609982,N/A,21.961814,N/A,N/A
4,ABBV,113.090,N/A,25.44,N/A,13.65,N/A,4.97,N/A,16.810105,N/A,9.977979,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.360,N/A,31.98,N/A,-4.18,N/A,5.95,N/A,24.241734,N/A,10.470988,N/A,N/A
501,ZBH,168.730,N/A,-223.41,N/A,2.98,N/A,4.9,N/A,28.278242,N/A,8.021952,N/A,N/A
502,ZBRA,414.070,N/A,47.04,N/A,11.45,N/A,5,N/A,32.698386,N/A,11.776553,N/A,N/A
503,ZION,49.150,N/A,21.06,N/A,1.12,N/A,2.63,N/A,11.540961,N/A,3.045305,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [22]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,217.990,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,240.530,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,12.000,N/A,10.09,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,33.400,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,49.910,N/A,14.47,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,53.940,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,31.030,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,161.430,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,1981.860,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
326,MYL,16.630,N/A,31.47,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [23]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [24]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [25]:

metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0       0.873267
1       0.154455
2        0.49802
3            0.8
4       0.506931
         ...    
500     0.649505
501    0.0257426
502     0.813861
503     0.431683
504     0.821782
Name: PE Percentile, Length: 505, dtype: object
0       0.738614
1      0.0475248
2       0.405941
3       0.960396
4       0.863366
         ...    
500    0.0435644
501      0.39505
502     0.835644
503     0.106931
504     0.928713
Name: PB Percentile, Length: 505, dtype: object
0       0.815842
1      0.0277228
2       0.164356
3       0.841584
4       0.694059
         ...    
500     0.744554
501     0.685149
502      0.69802
503     0.410891
504     0.936634
Name: PS Percentile, Length: 505, dtype: object
0      0.847525
1      0.029703
2      0.269307
3      0.809901
4      0.491089
         ...   
500    0.712871
501    0.792079
502    0.839604
503    0.277228
504    0.849505
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0      0.809901
1      0.049505
2      0.114851
3      0.948515


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,128.912,N/A,57.81,0.873267,8.10,0.738614,7.6700,0.815842,33.690997,0.847525,13.806188,0.809901,N/A
1,AAL,15.884,N/A,-1.16,0.154455,-1.77,0.0475248,0.4111,0.0277228,-5.919055,0.029703,1.379237,0.049505,N/A
2,AAP,169.186,N/A,25.23,0.49802,3.04,0.405941,1.1700,0.164356,11.500474,0.269307,2.485614,0.114851,N/A
3,AAPL,139.550,N/A,43.50,0.8,34.81,0.960396,8.1800,0.841584,29.609982,0.809901,21.961814,0.948515,N/A
4,ABBV,113.090,N/A,25.44,0.506931,13.65,0.863366,4.9700,0.694059,16.810105,0.491089,9.977979,0.659406,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.360,N/A,31.98,0.649505,-4.18,0.0435644,5.9500,0.744554,24.241734,0.712871,10.470988,0.673267,N/A
501,ZBH,168.730,N/A,-223.41,0.0257426,2.98,0.39505,4.9000,0.685149,28.278242,0.792079,8.021952,0.506931,N/A
502,ZBRA,414.070,N/A,47.04,0.813861,11.45,0.835644,5.0000,0.69802,32.698386,0.839604,11.776553,0.730693,N/A
503,ZION,49.150,N/A,21.06,0.431683,1.12,0.106931,2.6300,0.410891,11.540961,0.277228,3.045305,0.144554,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [27]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,128.912,N/A,57.81,0.873267,8.10,0.738614,7.6700,0.815842,33.690997,0.847525,13.806188,0.809901,0.81703
1,AAL,15.884,N/A,-1.16,0.154455,-1.77,0.0475248,0.4111,0.0277228,-5.919055,0.029703,1.379237,0.049505,0.0617822
2,AAP,169.186,N/A,25.23,0.49802,3.04,0.405941,1.1700,0.164356,11.500474,0.269307,2.485614,0.114851,0.290495
3,AAPL,139.550,N/A,43.50,0.8,34.81,0.960396,8.1800,0.841584,29.609982,0.809901,21.961814,0.948515,0.872079
4,ABBV,113.090,N/A,25.44,0.506931,13.65,0.863366,4.9700,0.694059,16.810105,0.491089,9.977979,0.659406,0.64297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.360,N/A,31.98,0.649505,-4.18,0.0435644,5.9500,0.744554,24.241734,0.712871,10.470988,0.673267,0.564752
501,ZBH,168.730,N/A,-223.41,0.0257426,2.98,0.39505,4.9000,0.685149,28.278242,0.792079,8.021952,0.506931,0.48099
502,ZBRA,414.070,N/A,47.04,0.813861,11.45,0.835644,5.0000,0.69802,32.698386,0.839604,11.776553,0.730693,0.783564
503,ZION,49.150,N/A,21.06,0.431683,1.12,0.106931,2.6300,0.410891,11.540961,0.277228,3.045305,0.144554,0.274257


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [28]:

rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AIG,42.383,N/A,-7.440000,0.112871,0.5737,0.0574257,0.8156,0.0752475,4.811884,0.0455446,0.801178,0.0158416,0.0613861
1,AAL,15.884,N/A,-1.160000,0.154455,-1.7700,0.0475248,0.4111,0.0277228,-5.919055,0.029703,1.379237,0.049505,0.0617822
2,UNM,26.791,N/A,5.540000,0.19802,0.4841,0.0514851,0.4506,0.0336634,2.895286,0.0356436,0.450052,0.0039604,0.0645545
3,F,11.820,N/A,-287.490000,0.0237624,1.3100,0.148515,0.3350,0.0138614,4.515815,0.039604,2.283674,0.106931,0.0665347
4,MET,51.725,N/A,8.450000,0.209901,0.6530,0.0613861,0.7208,0.0594059,4.759491,0.0435644,0.704290,0.0118812,0.0772277
5,FTI,11.395,N/A,-0.890000,0.156436,1.2600,0.133663,0.3909,0.0237624,3.441175,0.0376238,2.181626,0.10297,0.0908911
6,BA,212.880,N/A,-27.040000,0.0752475,-10.1000,0.0336634,2.0600,0.331683,-28.863906,0.0178218,-206.148309,0.0019802,0.0920792
7,ALL,114.760,N/A,8.150000,0.205941,1.3400,0.153465,0.7556,0.0633663,2.582886,0.0336634,0.749864,0.0138614,0.0940594
8,ABC,105.603,N/A,-6.280000,0.120792,-21.8200,0.019802,0.1142,0.0039604,8.230695,0.114851,3.950951,0.219802,0.0958416
9,HIG,52.620,N/A,10.840000,0.225743,1.0900,0.10099,0.9378,0.0970297,4.582282,0.0415842,0.933311,0.0237624,0.0978218


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [29]:
portfolio_input()

Enter the value of your portfolio: 10000000


In [30]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

/home/venom/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AIG,42.383,4718,-7.440000,0.112871,0.5737,0.0574257,0.8156,0.0752475,4.811884,0.0455446,0.801178,0.0158416,0.0613861
1,AAL,15.884,12591,-1.160000,0.154455,-1.7700,0.0475248,0.4111,0.0277228,-5.919055,0.029703,1.379237,0.049505,0.0617822
2,UNM,26.791,7465,5.540000,0.19802,0.4841,0.0514851,0.4506,0.0336634,2.895286,0.0356436,0.450052,0.0039604,0.0645545
3,F,11.820,16920,-287.490000,0.0237624,1.3100,0.148515,0.3350,0.0138614,4.515815,0.039604,2.283674,0.106931,0.0665347
4,MET,51.725,3866,8.450000,0.209901,0.6530,0.0613861,0.7208,0.0594059,4.759491,0.0435644,0.704290,0.0118812,0.0772277
5,FTI,11.395,17551,-0.890000,0.156436,1.2600,0.133663,0.3909,0.0237624,3.441175,0.0376238,2.181626,0.10297,0.0908911
6,BA,212.880,939,-27.040000,0.0752475,-10.1000,0.0336634,2.0600,0.331683,-28.863906,0.0178218,-206.148309,0.0019802,0.0920792
7,ALL,114.760,1742,8.150000,0.205941,1.3400,0.153465,0.7556,0.0633663,2.582886,0.0336634,0.749864,0.0138614,0.0940594
8,ABC,105.603,1893,-6.280000,0.120792,-21.8200,0.019802,0.1142,0.0039604,8.230695,0.114851,3.950951,0.219802,0.0958416
9,HIG,52.620,3800,10.840000,0.225743,1.0900,0.10099,0.9378,0.0970297,4.582282,0.0415842,0.933311,0.0237624,0.0978218


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [31]:

writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [32]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [33]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [34]:

writer.save()